### Ingestion del archivo "person.json"

### Paso 1 - Leer el archivo JSON usando "DataFrameReader" de Spark

In [0]:
dbutils.widgets.text("p_enviroment", "")
v_enviroment = dbutils.widgets.get("p_enviroment")

In [0]:
dbutils.widgets.text("p_file_date", "2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.functions import current_timestamp,lit,col,concat
from pyspark.sql.types import StructType,StructField,IntegerType,StringType


In [0]:
name_schema = StructType([
    StructField("forename", StringType(), True),
    StructField("surname", StringType(), True)
])

In [0]:
persons_schema = StructType([
    StructField("personId", IntegerType(), False),
    StructField("personName", name_schema)
    ])


In [0]:
persons_df = spark.read.schema(persons_schema).json(f"{bronze_folder_path}/{v_file_date}/person.json")

In [0]:
persons_df.printSchema()

In [0]:
display(persons_df)

#### Paso 2 - Renombrar las columnas y añadir nuevas columnas
1. "personid" renombrar a "person_id"
2. Agregar las columnas "ingestion_date" y "enviroment"
3. Agregar la columna "name" a partir de la concatenacion de "forename" y "surname"

In [0]:
persons_with_columns_df = add_ingestion_date(persons_df).withColumnRenamed("personId","person_id").withColumns({"enviroment":lit(v_enviroment),"file_date":lit(v_file_date)}).withColumn("name",concat(col("personName.forename"),lit(" "),col("personName.surname")))

In [0]:
display(persons_with_columns_df)

#### Paso 3 - Eliminar columnas no "requeridas"

In [0]:
persons_final_df = persons_with_columns_df.drop(col("personName"))

In [0]:
display(persons_final_df)

#### Paso 4 - Escribir la salida en un formato "Parquet"

In [0]:
merge_condition = 'tgt.person_id = src.person_id AND tgt.file_date = src.file_date'
merge_delta_lake(persons_final_df, 'movie_silver', 'persons', silver_folder_path, merge_condition, 'file_date')

In [0]:
%fs
ls /mnt/historialpeliculas/silver/persons/

In [0]:
%sql
SELECT file_date, COUNT(1)
FROM movie_silver.persons
GROUP BY file_date;

In [0]:
dbutils.notebook.exit("Success")